In [ ]:
""" 
setting up generic day analysis, iterate over multiple days

# TO DO
# 1) use generic HD to allow comparison across days
    ( this also allows beh analysis without affect of online changes in params)
# 2) Across-day analysi.


9/12/20 - Updated to plota ll trials, and to plot block-realted mean stats

9/20/20 -
MOVED TO tools/preprocess.py. Makes this obsolete
"""

In [1]:
%load_ext autoreload
%autoreload 2

%cd ..

/data1/code/python/drawmonkey


In [2]:
# TO DO
# 1) use generic HD to allow comparison across days
# 2) Across-day analysi.


In [3]:

from tools.utils import * 
from tools.plots import *
from tools.analy import *
from tools.calc import *
from tools.analyplot import *
from tools.preprocess import *
from tools.dayanalysis import *

from pythonlib.drawmodel.analysis import *
from pythonlib.tools.stroketools import *

NOTE: need to not overwrite strokes_all_task, because then the orders saved will stop being accurate. Modify


In [ ]:
# expt = "pilot"
# expt = "lines2"
AD = (
    ("Pancho", 200913, "lines4", 1),
    ("Pancho", 200914, "lines4", 1),
    ("Pancho", 200914, "lines4", 2),
    ("Red", 200913, "lines4", 1),
    ("Red", 200913, "lines4", 2),
    ("Red", 200914, "lines4", 1),
    ("Red", 200914, "lines4", 2),
    ("Red", 200914, "lines4", 3),
    ("Red", 200914, "lines4", 4),
)

for animal, date, expt, session in AD:
    fd = loadSingleData(animal, date, expt, session, resave_as_dict=False, load_resaved_data=True, 
                          resave_overwrite=False)
    
    if fd is None:
        continue
        
    # setup saving dir for figs
    SAVEDIRDAY = f"{fd['params']['figuredir']}/dayanalysis"
    import os
    os.makedirs(SAVEDIRDAY, exist_ok=True)

    # 1) overview
    df = extractSessionDf(fd)
    
    # ==== get features to plot.
    featurestoplot = []
    for key, val in getTrialsBlockParams(fd, 1)["behEval"]["beh_eval"].items():
        if val["weight"][0][0]>0:
            featurestoplot.append(val["feature"])
#         if val["feature"] =="hausdorff" and val["weight"][0][0]>0:
#             featurestoplot.append("hausdorff")
#         if val["feature"] =="frac_touched" and val["weight"][0][0]>0:
#             featurestoplot.append("frac_touched")
    featurestoplot.append("score_offline")
        
    fig1, fig2, fig3, fig4 = plotOverview_(df, featurestoplot=featurestoplot)
    fig1.savefig(f"{SAVEDIRDAY}/overview1.pdf")
    fig2.savefig(f"{SAVEDIRDAY}/overview2.pdf")
    fig3.savefig(f"{SAVEDIRDAY}/overview3.pdf")
    fig4.savefig(f"{SAVEDIRDAY}/overview4.pdf")
    plt.close()
    
    # 2) relationship between reward and factors that go into reward
    figs = plotReward(df, featurestoplot=featurestoplot)
    for i, f in enumerate(figs):
        f.savefig(f"{SAVEDIRDAY}/reward_score_{i}.pdf")

    # 3) PLOT BEHAVIOR FOR TRIALS SORTED BY SCORE
    import copy
    scoretypes = copy.copy(featurestoplot)
    scoretypes.extend(["behscore", "reward"])
    for score_type in scoretypes:
        FIGS = plotBehSortedByScore(df, fd, score_type)
        for ver, figs in FIGS.items():
            for i, f in enumerate(figs):
                f.savefig(f"{SAVEDIRDAY}/trialsSortedByScore_{score_type}_{ver}_{i}_.pdf")
            
    # 4) Plot behavior subsampling in chronological order
    trials = [t for t in getIndsTrials(fd) if getTrialsFixationSuccess(fd, t)]
    Nrand = 80
    fig = plotMultTrialsSimple(fd, trials, zoom=True, strokes_ver="peanuts", plot_fix=False, rand_subset=Nrand)
    fig.savefig(f"{SAVEDIRDAY}/trialsRandomChronOrder.pdf")

    # 5) ==== plot all trials in chron order
    trials = [t for t in getIndsTrials(fd) if getTrialsFixationSuccess(fd, t)]
    NplotPerFig = 80
    nfigs = int(np.ceil(len(trials)/NplotPerFig))
    for n in range(nfigs):
        if n==nfigs-1:
            idx = range(n*NplotPerFig, len(trials))
            trialsthis = [trials[i] for i in idx]
        else:
            idx = range(n*NplotPerFig, (n+1)*NplotPerFig)
            trialsthis = [trials[i] for i in idx]

        fig = plotMultTrialsSimple(fd, trialsthis, zoom=True, strokes_ver="peanuts", plot_fix=False)
        fig.savefig(f"{SAVEDIRDAY}/trialsAllChronOrder-{n}.pdf")

    # 5) TASK VISUALIZATIONS, SCHEDULE, REPETITION
    figs = plotTaskSchedules(df)
    for i, f in enumerate(figs):
        f.savefig(f"{SAVEDIRDAY}/taskSchedule{i}.pdf")
    plt.close()


In [ ]:
trials = [t for t in getIndsTrials(fd) if getTrialsFixationSuccess(fd, t)]
NplotPerFig = 10
nfigs = int(np.ceil(len(trials)/NplotPerFig))
for n in range(nfigs):
    if n==nfigs-1:
        idx = range(n*NplotPerFig, len(trials))
        trialsthis = [trials[i] for i in idx]
    else:
        idx = range(n*NplotPerFig, (n+1)*NplotPerFig)
        trialsthis = [trials[i] for i in idx]

    fig = plotMultTrialsSimple(fd, trialsthis, zoom=True, strokes_ver="peanuts", plot_fix=False)
    assert False

In [ ]:
df = extractSessionDf(fd)

# ==== get features to plot.
featurestoplot = []
for key, val in getTrialsBlockParams(fd, 1)["behEval"]["beh_eval"].items():
    if val["weight"][0][0]>0:
        featurestoplot.append(val["feature"])
#         if val["feature"] =="hausdorff" and val["weight"][0][0]>0:
#             featurestoplot.append("hausdorff")
#         if val["feature"] =="frac_touched" and val["weight"][0][0]>0:
#             featurestoplot.append("frac_touched")
featurestoplot.append("score_offline")

fig1, fig2 = plotOverview_(df, featurestoplot=featurestoplot)
# fig1.savefig(f"{SAVEDIRDAY}/overview1.pdf")
# fig2.savefig(f"{SAVEDIRDAY}/overview2.pdf")


In [ ]:
if False:
    animal = "Pancho"
    expt = "pilot"
    date = 200822
    session = 1

    fd = loadSingleData(animal, date, expt, session, resave_as_dict=False, load_resaved_data=True, 
                              resave_overwrite=False)

    ## ==== [testing] stroke based hd
    from pythonlib.tools.stroketools import distanceDTW
    t = random.sample(getIndsTrials(fd),1)[0]

    for ass in [True, False]:
        print(f"assymetric: {ass}")
        plotTrialSimple(fd, t, zoom=True, plot_fix=False, plotver="strokes", 
                        use_peanut_params={'replaynum': 1, 'active': True})

        strokes_beh = getTrialsStrokesByPeanuts(fd, t, replaynum=1)
        strokes_task = getTrialsTaskAsStrokes(fd, t)
        print(distanceDTW(strokes_beh, strokes_task[::-1], ver="segments", asymmetric=ass))
        print(distanceDTW(strokes_beh, strokes_task, ver="segments", asymmetric=ass))

        plt.figure(figsize=(10,10))
        ax = plt.subplot(211)
        plotDatStrokes(strokes_beh, ax=ax)
        plotDatStrokes(strokes_task[::-1], ax=ax)

        ax = plt.subplot(212)
        plotDatStrokes(strokes_beh, ax=ax)
        plotDatStrokes(strokes_task, ax=ax)


In [ ]:
## plotting distrubtions for all beh eval factors within the day
animal = "Pancho"
expt = "lines2"
date = 200904
session = 2


fd = loadSingleData(animal, date, expt, session, resave_as_dict=False, load_resaved_data=True, 
                          resave_overwrite=False)


In [ ]:
%matplotlib notebook
plotTrialSingleOverview(fd, 170);

In [ ]:
getTrialsOutcomesWrapper(fd, 170)

In [ ]:
[plotTrialSimple(fd, t) for t in range(102, 106)]

In [ ]:
X = [getTrialsOutcomesWrapper(fd, t)["beh_evaluation"]["output"]["dist_total"]["value"][0][0] for t in getIndsTrials(fd) if getTrialsFixationSuccess(fd, t)]

In [ ]:
plt.figure()
plt.hist(X, 50)

In [ ]:
plt.figure()
plt.hist(X, 50)

In [ ]:
[print(f"{t} - {getTrialsBlockParamsHotkeyUpdated(fd, t)['task_staging']['task_scheduler']}") for t in getIndsTrials(fd)]


In [ ]:
# === save trials rpograms so can resynthesize in matlab task

getTrialsTask(fd, 1)["TaskNew"]["Task"]["program"]


# RPobably best way is to save a list of trial numbers, open those in matlab, then save those tasks.

## TO DO:
(1) get all permutations
(2) normalize by num strokes.
(3) systematic - compare to old version.